In [102]:
import configparser
config = configparser.ConfigParser()
config.read('../training/secrets.ini')
import os
import openai
os.environ.update({'OPENAI_API_KEY': config['OPENAI']['OPENAI_API_KEY']})
openai.api_key = os.getenv('OPENAI_API_KEY')

In [103]:
system_prompt_template= """
You are an expert on sparql and wikibase. I have a private wikibase where the p and q items are completely unknown to you. 

If you are asked to provide a "query template json" for a user question, you should respond with a json object with these keys and values:
* 'query_template': a python-style template string. the query template should not use 'rdfs:label' to help you find items. Instead, just use a placeholder tag for each p and q item you need. Each tag should begin with "p-" or "q-" depending on whether you think you need a p or a q item
* 'vocabulary': a list of objects, one for each p or q item you need for a sparql query, each item should have these components:
** 'item_tag': a placeholder string that you make up
** 'item_label_quesses': a list of up to three different strings that you think might be used as the label for that p or q item
Remember: you do not know the actual q or p items in my wikibase, so make sure to list ALL p and q items in the 'vocabulary' section of the json.

You may assume the following prefixes:
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX p: <http://www.wikidata.org/prop/>
PREFIX ps: <http://www.wikidata.org/prop/statement/>

Respond only with the json. Do not include any comments or explanations.

"""

In [104]:
example1_human = """
generate a query template json for the question "Does malin 1 have a right ascension lower than 15.1398?"
"""

In [105]:
example1_ai = """
{{
  "query_template": "SELECT ?result WHERE {{ wd:q-Malin1 wdt:p-RightAscension ?right_ascension . FILTER(?right_ascension < 15.1398) BIND(xsd:boolean(?right_ascension < 15.1398) as ?result) }}",
  "vocabulary": [
    {{
      "item_tag": "q-Malin1",
      "item_label_quesses": ["Malin 1", "Malin-1", "Malin1"]
    }},
    {{
      "item_tag": "p-RightAscension",
      "item_label_quesses": ["Right ascension", "right_ascension", "RA"]
    }}
  ]
}}
"""

In [106]:
example2_human = """
How many children did J.S. Bach have?
"""

In [107]:
example2_ai = """
{{
  "query_template": "w
  "vocabulary": [
    {{
      "item_tag": "q-JSBach",
      "item_label_quesses": ["Johann Sebastian Bach", "Bach, Johann Sebastian", "J.S. Bach"]
    }},
    {{
      "item_tag": "p-Child",
      "item_label_quesses": ["Child", "Offspring", "Progeny"]
    }}
  ]
}}
"""

In [108]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [109]:
template=system_prompt_template
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

example_human_1 = HumanMessagePromptTemplate.from_template(example1_human)
example_ai_1 = AIMessagePromptTemplate.from_template(example1_ai)
example_human_2 = HumanMessagePromptTemplate.from_template(example2_human)
example_ai_2 = AIMessagePromptTemplate.from_template(example2_ai)

human_template="Please generate query template json for this question: {text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [110]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(temperature=0)

In [111]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, example_human_1, example_ai_1, example_human_2, example_ai_2, human_message_prompt])

In [112]:
from langchain import PromptTemplate, LLMChain
chain = LLMChain(llm=chat, prompt=chat_prompt)
result = chain.run('Which is the Basketball-Reference.com NBA player ID of Hakeem Olajuwon?')
print(result)

{
  "query_template": "SELECT ?player_id WHERE { wd:q-HakeemOlajuwon wdt:p-BasketballReferencecomNBAPlayerID ?player_id . }",
  "vocabulary": [
    {
      "item_tag": "q-HakeemOlajuwon",
      "item_label_quesses": ["Hakeem Olajuwon", "Olajuwon, Hakeem"]
    },
    {
      "item_tag": "p-BasketballReferencecomNBAPlayerID",
      "item_label_quesses": ["Basketball-Reference.com NBA player ID", "BasketballReference.com NBA Player ID", "Basketball-Reference NBA player ID"]
    }
  ]
}


In [12]:
import json

In [13]:
query_template = json.loads(result)

In [14]:
from typing import Any, Dict, List, Optional, Union

def get_nested_value(nested_dict: Dict[str, Any], path: List[str]) -> Optional[Any]:
    """
    Retrieves a value from a nested dictionary structure using a list of keys as a path.

    :param nested_dict: The nested dictionary structure to traverse.
    :param path: A list of strings representing the keys to traverse in order to access the desired value.
    :return: The value at the end of the path if it exists, or None if any key in the path is not found.
    """
    current = nested_dict
    for key in path:
        if not isinstance(current, dict) or key not in current:
            return None
        current = current[key]
    return current

In [15]:
import requests
from typing import Optional

def vocab_lookup(search: str, entity_type: str = "item",
                 item_tag: str = None, 
                 srqiprofile: str = "classic") -> Optional[str]:
    
    if item_tag is not None:
        if item_tag.startswith("q-"):
            entity_type = "item"
        elif item_tag.startswith("p-"):
            entity_type = "property"   
    
    if entity_type == "item":
        srnamespace = 0
    elif entity_type == "property":
        srnamespace = 120
    else:
        raise ValueError("entity_type must be either 'property' or 'item'")      
        
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": search,
        "srnamespace": srnamespace,
        "srlimit": 5,
        "srqiprofile": srqiprofile,
        "format": "json"
    }
    headers = {'Accept': 'application/json'}

    response = requests.get(url, headers=headers, params=params)
        
    if response.status_code == 200:
        results = get_nested_value(response.json(), ['query', 'search'])
        if results and len(results) > 0:
            title = results[0]['title']
            # properties are returned with a prefix 'property:'
            return results[0]['title'].split(':')[-1]
        else:
            print(f"No {entity_type} found for '{search}'")
            return ""
    else:
        print(f"Request failed with status code {response.status_code}")
        return ""

In [99]:
def resolve_query_template(t):
    vocab_items = [(v["item_tag"], v["item_label_quesses"]) for v in t["vocabulary"]]
    # since this is not a proper template, we use "replace" below
    # so we need to be careful that some tags may be prefixes of others
    # sorting them by tag length avoids that
    vocab_items = sorted(vocab_items, key=lambda x: x[0], reverse=True)
    resolved_vocabulary = {item_tag: vocab_lookup(item_label_quesses[0], item_tag=item_tag) for item_tag, item_label_quesses in vocab_items}
    query = t['query_template']
    for item_tag, item_id in resolved_vocabulary.items():
        query = query.replace(item_tag, item_id)
    result = query
    return result

In [17]:
resolve_query_template(query_template)

'SELECT ?player_id WHERE { wd:Q273256 wdt:P2685 ?player_id . }'

In [18]:
def translator(question):
    llm_result = chain.run(question)
    try:
        query_template = json.loads(llm_result)
    except Exception:
        query_template = "" 
    query = resolve_query_template(query_template)
    return query


In [19]:
translator("The Juventus F.C. has what number of participating teams?")

'SELECT (COUNT(?team) as ?result) WHERE { wd:Q1422 wdt:P1923 ?team }'

# Validation testing for Natural Language Question to SPARQL Query

Create a df for the lcquad queries

In [20]:
import pandas as pd

In [21]:
lcquad_path = "../../lcquad2.0.train.json"

In [22]:
!ls -l {lcquad_path}

-rw-r--r--@ 1 i857913  staff  26561843 Feb  6 16:58 ../../lcquad2.0.train.json


In [23]:
lcquad_df = pd.read_json(lcquad_path)

In [24]:
len(lcquad_df)

24180

Make a sparql runner

In [25]:
from wikibaseintegrator import wbi_helpers
from wikibaseintegrator.wbi_config import config as wbi_config
import logging

In [26]:
wbi_config['USER_AGENT'] = 'AskwikiBot/1.0 (https://www.wikidata.org/wiki/User:What_Tottles_Meant)'
wbi_config['BACKOFF_MAX_TRIES'] = 1


In [57]:
from requests.exceptions import HTTPError

def run_sparql(query):
    try:
        results = wbi_helpers.execute_sparql_query(query)
    except HTTPError as he:
        logging.error(f'HTTPError {he}')
        print(f"failed query {query}")
        return None
    # print(results)
    if 'boolean' in results:
        return pd.DataFrame([{'Boolean': results['boolean'] }])
    jsonResult = [dict([(k, b[k]['value']) for k in b]) for b in results['results']['bindings']]
    df = pd.DataFrame.from_dict(jsonResult)
    return df


In [67]:
import requests
from typing import List, Dict, Any

def run_sparql(query: str) -> List[Dict[str, Any]]:
    url = 'https://query.wikidata.org/sparql'
    headers = {
        'Accept': 'application/json',
        'User-Agent': 'AskwikiBot/1.0 (https://www.wikidata.org/wiki/User:What_Tottles_Meant)'
    }

    response = requests.get(url, headers=headers, params={'query': query, 'format': 'json'})

    if response.status_code == 200:
        results = response.json().get('results', {}).get('bindings', [])
        return results
    else:
        print(f"Request failed with status code {response.status_code}")
        return None


In [68]:
import time
def validate_queries(qs):
    validation_results = []
    count = 0
    for q in qs:
        print(count)
        if not 'limit' in q.lower():
            q += ' limit 100'
        print(q)
        df = run_sparql(q)
        result_count = 0
        if df is None:
            run_result = 'Fail'
            print(f"Failed query number {count}")
        else:
            run_result = 'Pass'
            result_count = len(df)
        validation_results.append((run_result, result_count))
        count += 1
        time.sleep(2)
    return validation_results 

In [69]:
v = validate_queries(lcquad_df['sparql_wikidata'].loc[0:5])

0
 select distinct ?obj where { wd:Q188920 wdt:P2813 ?obj . ?obj wdt:P31 wd:Q1002697 }  limit 100
1
SELECT ?answer WHERE { wd:Q169794 wdt:P26 ?X . ?X wdt:P22 ?answer} limit 100
2
ASK WHERE { wd:Q174843 wdt:P106 wd:Q1804811 . wd:Q174843 wdt:P106 wd:Q33231 } limit 100
Request failed with status code 400
Failed query number 2
3
SELECT ?answer WHERE { wd:Q675176 wdt:P515 ?X . ?X wdt:P156 ?answer} limit 100
4
select distinct ?answer where { wd:Q32491 wdt:P3362 ?answer} limit 100
5
SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj wdt:P31 wd:Q134041 . ?sbj rdfs:label ?sbj_label . FILTER(STRSTARTS(lcase(?sbj_label), 'p')) . FILTER (lang(?sbj_label) = 'en') } LIMIT 25 


In [30]:
v_df = pd.DataFrame(v)

In [31]:
len(v_df[v_df[0] == 'Pass'])

5

    
## Validate the queries

In [32]:
def generate_sparql(question):
    sparql = translator(f"translate English to SPARQL: {question}")
    logging.info(f'sparql {sparql}')
    return sparql


In [33]:
generate_sparql('What are the most common types of liver infection')

"SELECT ?type (COUNT(?disease) AS ?count) WHERE { ?disease wdt:P31 ?type . ?disease rdfs:label ?label . FILTER(LANG(?label) = 'en') FILTER(CONTAINS(LCASE(?label), 'liver infection')) } GROUP BY ?type ORDER BY DESC(?count) LIMIT 10"

In [34]:
def generate_lots_of_sparql(l):
    result = []
    count = 0
    for s in l:
        # if count % 10 == 0:
        print(s) 
        print(count)
        try:
            sp = generate_sparql(s)
        except Exception:
            sp = ""
        result.append(sp)
        count += 1
    print(count)
    return result


In [35]:
%%time
queries = generate_lots_of_sparql(lcquad_df['question'].loc[50:549])

Which country has highest individual tax rate?
0
When did James Thomas Farrell receive the St. Louis Literary Award?
1
When was Pablo Picasso's partnership with Fernade Oliver over?
2
What is the product with the lowest consumption rate per capita whose has part is paraffin wax?
3
No item found for 'bd:serviceParam'
What are the coordinates for the geographic center of Michigan, as determined by the center of gravity of the surface?
4
Is the life expectancy of Indonesia 55.3528?
5
When did Emmerson Mnangagwa begin going to the University of Zambia?
6
What kind of career does Grigori Kozintsev have in the screenwriting field?
7
what is hypothetical protein ecia139-4120 habitat of found in species?
8
What sovereign state replaced the Kingdom of Great Britain?
9
Of the century breaks of the Colm Gilcreest equal less than 9.6?
10
In which country the Golden Horde used to live ?
11
Located in the Central District, what is the county seat whose twin cities include Feodosiya?
12
No item found

In [36]:
len(queries)

500

In [40]:
serialized_list = json.dumps(queries)

with open("gpt3_turbo_queries.json", "w") as output_file:
    output_file.write(serialized_list)


In [71]:
%%time
v = validate_queries(queries)

0
SELECT ?country ?tax_rate WHERE { ?country wdt:P31 wd:Q6256 . ?country wdt:P297 ?iso_code . ?country wdt:P710 ?tax . ?tax wdt:P17 ?country . ?tax wdt:P2 ?tax_type . FILTER(?tax_type = wd:Q1056898) BIND(xsd:decimal(?tax) as ?tax_rate) } ORDER BY DESC(?tax_rate) LIMIT 1
1
SELECT ?date WHERE { ?Q51929074 rdfs:label "James Thomas Farrell"@en . ?Q51929074 wdt:Q43380 ?Q618779 . ?Q618779 rdfs:label "St. Louis Literary Award"@en . ?Q618779 wdt:Q47150325 ?date . } limit 100
2
SELECT ?end_date WHERE { wd:Q5593 wdt:P451 ?partner . ?partner wdt:P582 ?end_date . ?partner rdfs:label 'Fernande Olivier'@en . } limit 100
3
SELECT ?productLabel WHERE { ?product wdt:P31 wd:Q542929 . ?product wdt:Q70339049 ?consumptionRate . ?product wdt:Q96775080* wd:QINSERT . FILTER NOT EXISTS { ?product wdt:Q96775080 ?subProduct . ?subProduct wdt:Q70339049 ?subConsumptionRate . FILTER(?subConsumptionRate < ?consumptionRate) } } ORDER BY ?consumptionRate LIMIT 1 SERVICE wikibase:label {  wikibase:language 'en'. ?produ

In [72]:
len(v)


500

In [73]:
v_df = pd.DataFrame(v)

In [74]:
len(v_df[v_df[0] == 'Pass'])

384

In [83]:
q = 'SELECT ?facilityLabel WHERE { ?defensive_wall wdt:P846 ?facility . SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . } }'

In [75]:
v_df

,0,1
0,Pass,0
1,Pass,0
2,Pass,0
3,Fail,0
4,Pass,0
...,...,...
495,Pass,0
496,Fail,0
497,Pass,0
498,Pass,0


In [76]:
translator("What is the product with the lowest consumption rate per capita whose has part is paraffin wax?")

TypeError: string indices must be integers

In [77]:
question = "What is the product with the lowest consumption rate per capita whose has part is paraffin wax?"

In [87]:
question = """
When did James Thomas Farrell receive the St. Louis Literary Award?",
"""

In [89]:
query_template_json = chain.run(question)
print(query_template_json)

{
  "query_template": "SELECT ?date WHERE { wd:q-JamesThomasFarrell wdt:p-Award ?award . ?award wdt:p-AwardReceivedDate ?date . ?award wdt:p-AwardReceived ?received . ?received rdfs:label \"St. Louis Literary Award\"@en }",
  "vocabulary": [
    {
      "item_tag": "q-JamesThomasFarrell",
      "item_label_quesses": ["James Thomas Farrell", "Farrell, James Thomas"]
    },
    {
      "item_tag": "p-Award",
      "item_label_quesses": ["Award", "Prize", "Honour"]
    },
    {
      "item_tag": "p-AwardReceivedDate",
      "item_label_quesses": ["Award received date", "Date of receiving award", "Date received"]
    },
    {
      "item_tag": "p-AwardReceived",
      "item_label_quesses": ["Award received", "Receiving award", "Award recipient"]
    }
  ]
}


In [100]:
query_template = json.loads(query_template_json)
resolve_query_template(query_template)

No property found for 'Award received date'


'SELECT ?date WHERE { wd:Q1371154 wdt:P166 ?award . ?award wdt: ?date . ?award wdt:P166 ?received . ?received rdfs:label "St. Louis Literary Award"@en }'

In [97]:
def resolve_query_template(t):
    vocab_items = [(v["item_tag"], v["item_label_quesses"]) for v in t["vocabulary"]]
    vocab_items = sorted(vocab_items, key=lambda x: x[1], reverse=True)
    print(vocab_items)
    resolved_vocabulary = {item_tag: vocab_lookup(item_label_quesses[0], item_tag=item_tag) for item_tag, item_label_quesses in vocab_items}
    print(resolved_vocabulary)
    query = t['query_template']
    for item_tag, item_id in resolved_vocabulary.items():
        query = query.replace(item_tag, item_id)
    result = query
    return result

In [85]:
query_template['vocabulary']

TypeError: string indices must be integers

In [86]:
'limit' in q.lower()

False